<a href="https://colab.research.google.com/github/romapavelko01/FluencyGecUCUDiploma/blob/main/MBart100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/asivokon/unlp-2023-shared-task.git

Cloning into 'unlp-2023-shared-task'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 125 (delta 76), reused 98 (delta 62), pack-reused 0
Receiving objects: 100% (125/125), 4.69 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
!python --version

Python 3.10.11


In [ ]:
def change_to3_8():
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 4
  !sudo apt install python3-pip
  !sudo apt-get install python3-distutils
  !sudo apt-get install python3-apt
  !sudo apt-get install --reinstall python3-distutils
  !sudo apt-get update

update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in auto mode


In [ ]:
!python --version

Python 3.8.10


In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.8 MB/s 


# Introduction

This notebook cover inference and fine-tuning of raw MBart100 on the task of GEC-fluency.

# Loading unlp-2023-shared-task's data

In [ ]:
path = 'unlp-2023-shared-task/data/gec-fluency/valid.src.txt'
true = open(path).read()
true_lines = true.split('\n')
true[:100]

'# 0002\nНаступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і'

# Raw MBart100 inference on errorful data and metrics collection

Sample prediction

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

uk_text = "Це речення містить в собі помилку"


model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

model = model.to('cuda')

# translate Ukrainian to Ukrainian
tokenizer.src_lang = "uk"
encoded_uk = tokenizer(uk_text, return_tensors="pt").to('cuda')
generated_tokens = model.generate(**encoded_uk, forced_bos_token_id=tokenizer.get_lang_id("uk"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Ця фраза містить в собі помилку.']

## Running batch inference on all dataset

In [ ]:
from tqdm import tqdm

generated_raw = []
for i in tqdm(range(0, len(true_lines), 4)):
  batch = true_lines[i:i+4]
  encoded_uk = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
  generated_tokens = model.generate(**encoded_uk, forced_bos_token_id=tokenizer.get_lang_id("uk"))
  generated_lines = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  generated_raw.append(generated_lines)
  if i % 40 == 0:
    print(f"This step's {i} generated texts: ", generated_lines)

  0%|          | 0/377 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/377 [00:04<27:46,  4.43s/it]

This step's 0 generated texts:  ['# 0002', 'Наступного ранку рівно о одиннадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив лікаря піти і побачити Джанге Банк, який, здається, це був майор і дуже хворий чоловік.', '«Я не лікар», – сказав я: «Чому ти не йдеш до лікаря?»', '"Босс" - сказав він: "Доктор Хоскінс повинен проїхати 20 миль по країні, щоб побачити хворих людей.']


  3%|▎         | 11/377 [00:16<08:01,  1.32s/it]

This step's 40 generated texts:  ['— Це вже другий — подумав я.', 'Цього разу вже не налякався, тому що цей другий труп лежав на спині і не виявляв жодних ознак життя і руху.', '- Скшетуський просто додав ходу, щоб не закрилася голова.', 'Очерет ставав все густішим - що з одного боку давало безпечний сховище, з іншого значно ускладнювало похід.']


  6%|▌         | 21/377 [00:28<05:24,  1.10it/s]

...

 96%|█████████▌| 361/377 [07:04<00:11,  1.39it/s]

This step's 1440 generated texts:  ['Це саме те, що собі лякати завтра, за те, що змінили свою думку сьогодні :)', 'І логічно?', '# 0379', 'Час року, день-ночі, урожай - все в цьому світі циклічно.']


 98%|█████████▊| 371/377 [07:16<00:06,  1.11s/it]

This step's 1480 generated texts:  ['Наступного дня Ніт зателефонував мені з таксі.', '«Раз чи дві я взяв невірний слід, доктор, — визнав він.', '— Я трохи знайомий з методами детективів і я керував деякими з них, розраховуючи на своєчасне виїзд на Джолнса.', "Пістолет був.45-го калібру, тому я думав, що обов'язково знайду його на 45-й вулиці, де він збиратиме докази."]


100%|██████████| 377/377 [07:22<00:00,  1.17s/it]


In [ ]:
i = 0
with open('temp_raw_MBart100.txt', 'w') as f:
  for batch in generated_raw:
    for line in batch:
      if i != len(true_lines) - 1:
        f.write(line + '\n')
      else:
        f.write(line)
      i += 1
      
f.close()
generated_raw_txt = open('temp_raw_MBart100.txt').read()
len(generated_raw_txt.split('\n')), len(true_lines)

(1507, 1507)

In [ ]:
# import shutil
# # copying the file to google drive
# shutil.copyfile('temp_raw_MBart100.txt', '/content/drive/MyDrive/UNLP/RESULTS/temp_raw_MBart100.txt')

'/content/drive/MyDrive/UNLP/RESULTS/temp_raw_MBart100.txt'

# Checking back-translation data